# Multi-Agent Systems with Subagents

In this notebook, we'll explore how to build multi-agent systems using subagents. Subagents let you create specialized agents that work together to solve complex tasks.

## Setup

First, let's set up our environment:

In [32]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

✓ Notebook environment configured


In [33]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

✓ API key found (length: 108 characters)


## Helper Function

Let's create a helper to print messages cleanly:

In [34]:
def print_message(message):
    """Pretty print agent messages with subagent details."""
    msg_type = type(message).__name__
    
    if msg_type == "SystemMessage":
        pass  # Skip system messages
    
    elif msg_type == "AssistantMessage":
        # Check if this message is from inside a subagent
        if hasattr(message, 'parent_tool_use_id') and message.parent_tool_use_id:
            print("  [Inside subagent context]")
        
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                
                if block_type == "TextBlock":
                    print(f"Assistant: {block.text}")
                
                elif block_type == "ToolUseBlock":
                    # Show detailed info for Task tool (subagent invocation)
                    if block.name == "Task":
                        subagent_type = block.input.get('subagent_type', 'unknown')
                        description = block.input.get('description', '')
                        print(f"\n→ Invoking subagent: {subagent_type}")
                        if description:
                            print(f"  Task: {description}")
                    
                    # Show TaskOutput checks
                    elif block.name == "TaskOutput":
                        task_id = block.input.get('task_id', 'unknown')
                        print(f"\n→ Checking subagent output (task: {task_id})")
                    
                    else:
                        print(f"\n→ Using tool: {block.name}")
                        # Show tool parameters for web tools
                        if block.name in ["WebSearch", "WebFetch"] and hasattr(block, 'input'):
                            if 'query' in block.input:
                                print(f"  Query: {block.input['query']}")
                            if 'url' in block.input:
                                print(f"  URL: {block.input['url']}")
    
    elif msg_type == "UserMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                
                if block_type == "ToolResultBlock":
                    if block.is_error:
                        print(f"✗ Tool Error: {block.content}")
                    else:
                        content = str(block.content)
                        # Show if this is a subagent completion
                        if "agentId:" in content:
                            print(f"✓ Subagent completed")
                            # Extract and show agent ID
                            import re
                            match = re.search(r"agentId:\s*([a-f0-9-]+)", content)
                            if match:
                                print(f"  Agent ID: {match.group(1)}")
                        
                        # Show truncated results
                        if len(content) > 500:
                            content = content[:500] + "..."
                        print(f"Result: {content}")
    
    elif msg_type == "ResultMessage":
        if hasattr(message, 'total_cost_usd') and hasattr(message, 'duration_ms'):
            print(f"\nCost: ${message.total_cost_usd:.4f} | Time: {message.duration_ms/1000:.1f}s")

## What Are Subagents?

Subagents are separate agent instances that your main agent can spawn to handle specialized subtasks.

### Why Use Subagents?

- **Specialization**: Each agent has focused expertise
- **Context Isolation**: Keep specialized work separate
- **Tool Restrictions**: Control what each agent can do
- **Parallelization**: Run multiple agents concurrently

### How to Define Subagents

```python
from claude_agent_sdk import AgentDefinition

AgentDefinition(
    description="When to use this agent",
    prompt="The agent's specialized instructions",
    tools=["Read", "Grep"],  # Optional: restrict tools
    model="sonnet"           # Optional: choose model
)
```

**Important**: Include `"Task"` in the main agent's `allowed_tools` for delegation.

## Example: Blog Writing

Let's create a content creation pipeline with specialized agents:

### How Subagents Return Results

When a subagent completes its work:
1. The subagent returns its output (text, analysis, etc.)
2. The main agent retrieves it using `TaskOutput`
3. The main agent can use that output directly or pass it to another subagent
4. The final result is presented to the user by the main agent

Subagents don't write files or save output themselves - they return their work to the main agent.

In [35]:
from claude_agent_sdk import query, ClaudeAgentOptions, AgentDefinition

async def blog_writing_example():
    async for message in query(
        prompt="""Write a 400-word blog post about the benefits of remote work. 

IMPORTANT: Follow this sequence:
1. First, use the researcher agent to gather facts and statistics
2. Wait for the researcher to complete and return results
3. Then, use the writer agent to create the blog post based on the research

Do NOT invoke the writer until research is complete.""",
        options=ClaudeAgentOptions(
            allowed_tools=["Task"],
            permission_mode="bypassPermissions",
            agents={
                "researcher": AgentDefinition(
                    description="Research specialist. Use this agent to gather facts, statistics, and examples about any topic using web search.",
                    prompt="""You are a research specialist. Your job is to:
1. Search the web for current information on the given topic
2. Identify key themes and important facts
3. Gather relevant examples and statistics
4. Organize findings clearly
5. Return a concise research summary

Use WebSearch to find information.""",
                    tools=["WebSearch", "WebFetch"],
                    model="sonnet"
                ),
                
                "writer": AgentDefinition(
                    description="Content writer. Use this agent to write blog posts, articles, and other content based on research or information provided.",
                    prompt="""You are a professional blog writer. Create engaging content with:
- Strong opening hook
- Clear structure (intro, body, conclusion)
- Conversational and accessible tone
- Specific examples and details
- Easy readability

Write based on the research provided to you. Save your blog post to a file.""",
                    tools=["Write"],
                    model="sonnet"
                )
            }
        )
    ):
        print_message(message)

await blog_writing_example()

Assistant: I'll help you create a well-researched blog post about the benefits of remote work. Let me start by gathering facts and statistics.

→ Invoking subagent: researcher
  Task: Research remote work benefits
  [Inside subagent context]

→ Using tool: WebSearch
  Query: remote work productivity statistics 2024 2025 2026
  [Inside subagent context]

→ Using tool: WebSearch
  Query: remote work employee satisfaction retention data 2024 2025
  [Inside subagent context]

→ Using tool: WebSearch
  Query: remote work cost savings employers employees 2024 2025
  [Inside subagent context]

→ Using tool: WebSearch
  Query: remote work environmental benefits carbon emissions 2024 2025
  [Inside subagent context]

→ Using tool: WebSearch
  Query: work-life balance remote work benefits 2024 2025 2026
Result: Web search results for query: "remote work employee satisfaction retention data 2024 2025"

Links: [{"title":"The rise in remote work since the pandemic and its impact on productivity : B